In [12]:
import pandas as pd
import numpy as np
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', None)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate
    while True:
        print("\n***** Shape: ", df.shape," *****\n")
        
        columns_list = df.columns.values.tolist()
        isnull_list = df.isnull().sum().values.tolist()
        isunique_list = df.nunique().values.tolist()
        dtypes_list = df.dtypes.tolist()
        
        list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
        df_stat_val = pd.DataFrame(list_stat_val, columns=['Name', 'Null', 'Unique', 'Dtypes'])
        print(tabulate(df_stat_val, headers='keys', tablefmt='psql'))
    return df.head()

/tmp/ipykernel_3179884/2808397451.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [13]:
# Load the specified CSV file
csv_file_path = '/media/data1/ravram/DeepCORO/processed_dataframes/ObjectRecon_SWIN3D_2016-2023_inference_predictions_with_df_metadata_and_report.csv'
df_predictions = pd.read_csv(csv_file_path)
display(df_predictions.head())

/tmp/ipykernel_3179884/869894219.py:3: DtypeWarning: Columns (4,5,10,23,27,33,34,38,43,45,46,73,80,81,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df_predictions = pd.read_csv(csv_file_path)


,Unnamed: 0,filename,y_hat,object_value,brand,sex,FPS,NumberOfFrames,date,study_time,series_time,birthdate,color_format,StudyID,StudyInstanceUID,SeriesInstanceUID,dicom_path,FileName,uint16_video,primary_angle,secondary_angle,width,height,pixel_spacing,distance_source_to_detector,distance_source_to_patient,estimated_radiographic_magnification_factor,table_motion,radiation_setting,image_pixel_spacing,Split,CathReport_MRN,EXAMEN_ID,Num Accession,date/heure,DICOMPath,AccessionNumber,ModalitiesInStudy,Patient_ID,QueryRetrieveLevel,StudyDate,StudyTime,year,patient_id_anon,dicom_id,Recommendation,Conclusion,D2,D3,RVG1,RVG2,S1,bx,diagonal,dist_lad,dist_lcx,dist_rca,lad,lcx,leftmain,lima_or_svg,lvp,marg_d,mid_lad,mid_rca,om1,om2,om3,pda,posterolateral,prox_rca,fps,frame_time,series_description,External_Exam,angle_value,Unnamed: 0_y,coronary_dominance_logit,coronary_dominance,acute_coronary_occlusion,pci_regions,pontage,acute_inferior_mi,Examen,G_INDICATIONS,ACS,insuffisance_cardiaque,cardiogenic_shock
0,0,/media/data1/ravram/MHI_CATH_DICOM_VIDEOS/2022/2.16.124.113611.1.118.1.1.5994023_1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000.dcm.avi,[1.4491844e-05 2.4803728e-03 1.6874110e-06 1.3522274e-04 1.2590732e-05\n 5.2861782e-04 1.6547735e-06 5.2691407e-06 9.0157828e-06 9.9675262e-01\n 5.8418289e-05],9,NaN,NaN,10.0,75.0,20220101,24927.0,33910.0,19580211.0,MONOCHROME2,'1.3.12.2.1107.5.4.5.135214.30000022010107451244200000012.512',2.16.124.113611.1.118.1.1.5994023,'1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000',/media/data1/ravram/DeepCORO/2022/68509394433809345732708154541135422799/2.16.124.113611.1.118.1.1.5994023/1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000.dcm,/media/data1/ravram/MHI_CATH_DICOM_VIDEOS/2022/2.16.124.113611.1.118.1.1.5994023_1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000.dcm.avi,False,30.0,-0.1,512,512,NaN,1078.0,732.997649,1.470673,NaN,GR,"[0.278875, 0.278875]",inference,601040.0,102859.0,HE202200000101,2022-01-01 03:35:00.0,/media/data1/ravram/DeepCORO/2022/68509394433809345732708154541135422799/2.16.124.113611.1.118.1.1.5994023/1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000.dcm,HE202200000101,XA,601040,STUDY,20220101,24927.0,2022,68509394433809345732708154541135422799,1.3.12.2.1107.5.4.5.135214.30000022010107492025500000000,1. Aspirine à vie.,"Fraction d'éjection du ventricule gauche normale. Artères coronaires atheromateuses. Lésion peu sévère de l'artère interventriculaire antérieure proximale, englobant l'origine de la première diagonale (lésion de bifurcation). Lésion peu sévère de l'ostium de l'artère interventriculaire antérieure proximale. Lésion peu sévère de l'artère circonflexe proximale. Lésion peu sévère de l'ostium de la première marginale. Lésion peu sévère de l'artère coronaire droite proximale, englobant l'origine de l'artère interventriculaire postérieure (lésion de bifurcation).",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,100.0,CORONAROGRAPHIE,False,5.0,0.0,[0.0003176],right_dominant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,/media/data1/ravram/MHI_CATH_DICOM_VIDEOS/2022/2.16.124.113611.1.118.1.1.5994023_1.3.12.2.1107.5.4.5.135214.30000022010107451244200000015.dcm.avi,[3.4116832e-07 4.8741687e-05 1.4333554e-07 5.5439459e-06 5.1315556e-07\n 1.9034522e-04 6.1296834e-07 1.4616259e-07 9.2558201e-07 9.9973506e-01\n 1.7579954e-05],9,NaN,NaN,10.0,64.0,20220101,24927.0,33931.0,19580211.0,MONOCHROME2,'1.3.12.2.1107.5.4.5.135214.30000022010107451244200000014.512',2.16.124.113611.1.118.1.1.5994023,'1.3.12.2.1107.5.4.5.135214.30000022010107451244200000015',/media/data1/ravram/DeepCORO/2022/68509394433809345732708154541135422799/2.16.124.113611.1.118.1.1.5994023/1.3.12.2.1107.5.4.5.135214.30000022010107451244200000015.dcm,/media/data1/ravram/MHI_CATH_DICOM_VIDEOS/2022/2.16.124.113611.1.118.1.1.5994023_1.3.12.2.1107.5.4.5.135214.30000022010107451244200000015.dcm.avi,False,0.1,22.6,512,512,NaN,1060.0,735.83111

In [14]:
import pandas as pd
from tqdm import tqdm

# Filtered labels to vessel names mapping
labels_to_vessel_names = {
    "lad": "Left Anterior Descending",
    "dist_lad": "Distal Left Anterior Descending",
    "mid_lad": "Mid Left Anterior Descending",
    "lcx": "Left Circumflex",
    "dist_lcx": "Distal Left Circumflex",
    "leftmain": "Left Main Coronary Artery",
    "prox_rca": "Proximal Right Coronary Artery",
    "mid_rca": "Mid Right Coronary Artery",
    "dist_rca": "Distal Right Coronary Artery",
    "posterolateral": "Posterolateral",
    "pda": "Posterior Descending Artery",
}

# Function to create the report for each row
def create_report(row):
    report_lines = []
    
    # Combine percentages with vessel names for the selected labels
    for label, vessel_name in labels_to_vessel_names.items():
        if label in row:
            percentage = row[label]
            if pd.notna(percentage) and percentage != -1:
                report_line = f"{vessel_name}: {percentage}%"
                report_lines.append(report_line)
    
    # Add coronary dominance
    if 'coronary_dominance' in row:
        report_lines.append(f"Coronary Dominance: {row['coronary_dominance']}")
    
    # Summarize Conclusion and Recommendation
    if 'Conclusion' in row:
        report_lines.append(f"Conclusion: {row['Conclusion']}")
    if 'Recommendation' in row:
        report_lines.append(f"Recommendation: {row['Recommendation']}")
    
    # Join all lines into a single string
    report = '\n'.join(report_lines)
    return report

# Apply the function to create the 'Report' column with tqdm progress bar
tqdm.pandas(desc="Generating Reports")
df_predictions['Report'] = df_predictions.progress_apply(create_report, axis=1)


Generating Reports: 100%|██████████| 925207/925207 [00:53<00:00, 17313.91it/s]


In [15]:
df_non_nan_reports = df_predictions.dropna(subset=['Report','Conclusion','Recommendation'])
# Filter the dataframe to keep only rows where 'object_value' is 5 or 9
df_non_nan_reports = df_non_nan_reports[df_non_nan_reports['object_value'].isin([5, 9])]

display(df_non_nan_reports.Report.sample(10))
# Save the 'Report' column to a text file with 'α' as the separator
output_file_path = 'data/processed/reports/reports_with_alpha_separator.csv'
df_non_nan_reports.to_csv(output_file_path, sep='α', index=False, header=True)


535953                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Left Anterior Descending: 0.0%\nDistal Left Anterior Descending: 0.0%\nMid Left Anterior Descending: 0.0%\nLeft Circumflex: 0.0%\nDistal Left Circumflex: 0.0%\nLeft Ma

In [16]:
output_file_path = 'data/processed/reports/reports_with_alpha_separator.csv'
df = pd.read_csv(output_file_path, sep='α', engine='python')


In [17]:

# Sample 1000 rows from the dataframe
df_sampled = df.sample(n=1000, random_state=42)

# Add a new column 'split' with value 'train'
df_sampled['Split'] = 'train'

# Save the sampled dataframe to a new CSV file
output_sampled_file_path = 'data/processed/reports/reports_sampled_1000.csv'
df_sampled.to_csv(output_sampled_file_path, sep='α', index=False)

display(df_sampled.Split.value_counts())

Split
train    1000
Name: count, dtype: int64